Check Colab GPU

In [1]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# print(gpu_info)

Thu Apr 13 03:57:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Check Colab memory

In [2]:
# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 89.6 gigabytes of available RAM



## Imports

In [16]:
# Transformers installation
! pip install transformers datasets evaluate rouge_score

# Imports
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline, AdamW, get_linear_schedule_with_warmup
import evaluate
import numpy as np
import unicodedata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Log in to hugging face to upload upload model

In [4]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load BillSum dataset

Start by loading the smaller California state bill subset of the BillSum dataset from the Hugging Face Datasets library:

In [5]:
billsum = load_dataset("billsum", split="ca_test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc. Subsequent calls will reuse this data.


Split the dataset into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [6]:
billsum = billsum.train_test_split(test_size=0.2)

Then take a look at an example:

In [7]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 69432 of the Education Code is amended to read:\n69432.\n(a) Cal Grant Program awards shall be known as “Cal Grant A Entitlement Awards,” “Cal Grant B Entitlement Awards,” “California Community College Transfer Entitlement Awards,” “Competitive Cal Grant A and B Awards,” “Cal Grant C Awards,” and “Cal Grant T Awards.”\n(b) Maximum award amounts for students at independent institutions and for Cal Grant C and T awards shall be identified in the annual Budget Act. Maximum award amounts for Cal Grant A and B awards for students attending public institutions shall be referenced in the annual Budget Act.\n(c) (1) Notwithstanding subdivision (b), and subdivision (c) of Section 66021.2, commencing with the 2013–14 award year, the maximum tuition award amounts for Cal Grant A and B awards for students attending private for-profit and nonprofit postsecondary educational institutions shall be as follows

There are two fields that we want to use:

- `text`: the text of the bill which'll be the input to the model.
- `summary`: a condensed version of `text` which'll be the model target.

## Preprocess

The next step is to load a T5 tokenizer to process `text` and `summary`:

In [8]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


The preprocessing function we want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Use the keyword `text_target` argument when tokenizing labels.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [9]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

To apply the preprocessing function over the entire dataset, use Hugging Face Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. We can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [10]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate

Including a metric during training to help evaluate model's performance. For this task, we will load the ROUGE metric from the Hugging Face Evaluate library.

In [12]:
rouge = evaluate.load("rouge")

In [13]:
bleu = evaluate.load("bleu")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the ROUGE metric:

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    bleu_result = bleu.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_labels])

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    rouge_result["gen_len"] = np.mean(prediction_lens)
    rouge_result["bleu"] = bleu_result["bleu"]

    return {k: round(v, 4) for k, v in rouge_result.items()}

## Train

Load T5 with [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM):

In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the ROUGE metric and save the training checkpoint.
2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [17]:
def create_optimizer_and_scheduler(model, learning_rate):
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    num_training_steps = len(tokenized_billsum["train"])
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
    )
    return optimizer, lr_scheduler

In [18]:
# Change the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# Add a learning rate scheduler
num_train_steps = len(tokenized_billsum["train"])
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

# Experiment with different batch sizes and learning rates
batch_sizes = [8, 16, 32]
learning_rates = [2e-5, 5e-5, 1e-4]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        training_args = Seq2SeqTrainingArguments(
            output_dir=f"T5-small_finetuned_billsum_subset_model_bs{batch_size}_lr{learning_rate}",
            evaluation_strategy="epoch",
            learning_rate=learning_rate,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            weight_decay=0.01,
            save_total_limit=3,
            num_train_epochs=4,
            predict_with_generate=True,
            fp16=True,
            push_to_hub=True,
        )

        optimizer, lr_scheduler = create_optimizer_and_scheduler(model, learning_rate)

        trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_billsum["train"],
            eval_dataset=tokenized_billsum["test"],
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
            optimizers=(optimizer, lr_scheduler),
        )
        
        trainer.train()

        # training is completed, share the model to the Hub
        trainer.push_to_hub()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Cloning https://huggingface.co/alaahussein/T5-small_finetuned_billsum_subset_model_bs8_lr2e-05 into local empty directory.
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu
1,No log,2.621088,0.133100,0.044200,0.110700,0.110600,19.000000,0.000400
2,No log,2.449672,0.145500,0.054400,0.122400,0.122100,19.000000,0.000600
3,No log,2.379583,0.186100,0.085300,0.157700,0.157400,19.000000,0.000800
4,No log,2.340686,0.195200,0.091800,0.165200,0.165000,19.000000,0.000900


Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.68k [00:00<?, ?B/s]

Upload file runs/Apr13_04-03-46_8e3ea4a00660/events.out.tfevents.1681358633.8e3ea4a00660.3949.0:   0%|        …

...

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file runs/Apr13_04-25-26_8e3ea4a00660/events.out.tfevents.1681359932.8e3ea4a00660.3949.16:   0%|       …

Upload file training_args.bin:   0%|          | 1.00/3.68k [00:00<?, ?B/s]

Upload file runs/Apr13_04-25-26_8e3ea4a00660/1681359932.2252629/events.out.tfevents.1681359932.8e3ea4a00660.39…

To https://huggingface.co/alaahussein/T5-small_finetuned_billsum_subset_model_bs32_lr0.0001
   0dfb1db..6738bfa  main -> main

   0dfb1db..6738bfa  main -> main

To https://huggingface.co/alaahussein/T5-small_finetuned_billsum_subset_model_bs32_lr0.0001
   6738bfa..babab83  main -> main

   6738bfa..babab83  main -> main

